In [1]:
import pandas as pd
import yfinance as yf
from backtesting import Backtest
from src.strategies import MomentumTimeSeries, SmaCross


ticker = "UPST".upper()
period = "10y"
stock = yf.Ticker(ticker)
data = stock.history(period=period)

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [6]:
bt = Backtest(data, SmaCross, cash=10_000, commission=0.004)
stats = bt.run()
stats

Start                     2020-12-16 00:00...
End                       2024-02-23 00:00...
Duration                   1164 days 00:00:00
Exposure Time [%]                   65.917603
Equity Final [$]                 10366.346872
Equity Peak [$]                  30724.885637
Return [%]                           3.663469
Buy & Hold Return [%]               -17.74686
Return (Ann.) [%]                    1.138373
Volatility (Ann.) [%]               84.364123
Sharpe Ratio                         0.013494
Sortino Ratio                        0.022521
Calmar Ratio                          0.01603
Max. Drawdown [%]                  -71.016434
Avg. Drawdown [%]                  -11.397778
Max. Drawdown Duration      207 days 00:00:00
Avg. Drawdown Duration       31 days 00:00:00
# Trades                                    2
Win Rate [%]                             50.0
Best Trade [%]                       64.22901
Worst Trade [%]                      -36.8063
Avg. Trade [%]                    

In [3]:
bt.plot()

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Row(id='1496', ...)

In [7]:
%%time
stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


CPU times: user 1.52 s, sys: 9.85 ms, total: 1.53 s
Wall time: 1.53 s


Start                     2020-12-16 00:00...
End                       2024-02-23 00:00...
Duration                   1164 days 00:00:00
Exposure Time [%]                   87.640449
Equity Final [$]                 29520.118165
Equity Peak [$]                  78290.321774
Return [%]                         195.201182
Buy & Hold Return [%]               -17.74686
Return (Ann.) [%]                   40.573132
Volatility (Ann.) [%]              142.030537
Sharpe Ratio                         0.285665
Sortino Ratio                        0.748958
Calmar Ratio                         0.613712
Max. Drawdown [%]                  -66.111006
Avg. Drawdown [%]                  -13.238674
Max. Drawdown Duration      280 days 00:00:00
Avg. Drawdown Duration       32 days 00:00:00
# Trades                                   23
Win Rate [%]                        43.478261
Best Trade [%]                     113.195294
Worst Trade [%]                    -33.279131
Avg. Trade [%]                    

In [8]:
%%time
bt = Backtest(data, MomentumTimeSeries, cash=10_000, commission=0)
stats = bt.optimize(
    lookback=range(1,10,1),
    maximize='Equity Final [$]',
    # constraint=lambda param: param,
)
stats

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


CPU times: user 1.01 s, sys: 6.98 ms, total: 1.01 s
Wall time: 1.02 s


Start                     2020-12-16 00:00...
End                       2024-02-23 00:00...
Duration                   1164 days 00:00:00
Exposure Time [%]                   99.126092
Equity Final [$]                 97707.500828
Equity Peak [$]                 201189.406078
Return [%]                         877.075008
Buy & Hold Return [%]               -17.74686
Return (Ann.) [%]                  104.850955
Volatility (Ann.) [%]              421.816052
Sharpe Ratio                          0.24857
Sortino Ratio                        1.323976
Calmar Ratio                         1.142419
Max. Drawdown [%]                   -91.77981
Avg. Drawdown [%]                  -14.970639
Max. Drawdown Duration      752 days 00:00:00
Avg. Drawdown Duration       39 days 00:00:00
# Trades                                  794
Win Rate [%]                        52.015113
Best Trade [%]                      49.126798
Worst Trade [%]                    -57.971552
Avg. Trade [%]                    